# Exploring International Debt stats API
- Following this guide: https://worldbank.github.io/debt-data/api-guide/ids-api-guide-python-1.html

In [19]:
# Importing packages
import pandas as pd
import numpy as np
import datetime
import wbdata
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook" # use "pio.renderers" to see the default renderer

### Get all sources from the World Bank API

In [20]:
sources = requests.get("http://api.worldbank.org/v2/sources?per_page=100&format=json")
sourcesJSON = sources.json()
sourcesJSON

[{'page': '1', 'pages': '1', 'per_page': '100', 'total': '68'},
 [{'id': '1',
   'lastupdated': '2021-08-18',
   'name': 'Doing Business',
   'code': 'DBS',
   'description': '',
   'url': '',
   'dataavailability': 'Y',
   'metadataavailability': 'Y',
   'concepts': '3'},
  {'id': '2',
   'lastupdated': '2022-05-25',
   'name': 'World Development Indicators',
   'code': 'WDI',
   'description': '',
   'url': '',
   'dataavailability': 'Y',
   'metadataavailability': 'Y',
   'concepts': '3'},
  {'id': '3',
   'lastupdated': '2021-09-27',
   'name': 'Worldwide Governance Indicators',
   'code': 'WGI',
   'description': '',
   'url': '',
   'dataavailability': 'Y',
   'metadataavailability': 'Y',
   'concepts': '3'},
  {'id': '5',
   'lastupdated': '2016-03-21',
   'name': 'Subnational Malnutrition Database',
   'code': 'SNM',
   'description': '',
   'url': '',
   'dataavailability': 'Y',
   'metadataavailability': 'Y',
   'concepts': '3'},
  {'id': '6',
   'lastupdated': '2022-01-14',


### Get indicators for external debt

In [21]:
# Requesting the indicators for the topic External Debt
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=6")
indicatorsJSON = indicators.json()
indicatorsJSON

[{'page': 1, 'pages': 12, 'per_page': '50', 'total': 569},
 [{'id': 'BM.GSR.TOTL.CD',
   'name': 'Imports of goods, services and primary income (BoP, current US$)',
   'unit': '',
   'source': {'id': '6', 'value': 'International Debt Statistics'},
   'sourceNote': 'Imports of goods, services and primary income is the sum of goods imports, service imports and primary income payments. Data are in current U.S. dollars.',
   'sourceOrganization': 'International Monetary Fund, Balance of Payments Statistics Yearbook and data files.',
   'topics': [{'id': '3', 'value': 'Economy & Growth'},
    {'id': '20', 'value': 'External Debt'},
    {'id': '21', 'value': 'Trade'}]},
  {'id': 'BN.CAB.XOKA.CD',
   'name': 'Current account balance (BoP, current US$)',
   'unit': '',
   'source': {'id': '6', 'value': 'International Debt Statistics'},
   'sourceNote': 'Current account balance is the sum of net exports of goods and services, net primary income, and net secondary income. Data are in current U.S

In [22]:
# Print the total number of indicators
print("There are " + str(indicatorsJSON[0]["total"]) + " IDS indicators")

There are 569 IDS indicators


In [23]:
# Get all External Debt indicators, with a per_page parameter of 500.
indicators = requests.get("http://api.worldbank.org/v2/indicator?format=json&source=6&per_page=500")
indicatorsJSON = indicators.json()
#print(indicatorsJSON) # to view ALL of the indicators as is, remove the "#" at the beginning of the print command

# Parse through the response to see the Indicator IDs and Names
for i in indicatorsJSON[1]:
    IDSindicators = (i["id"],i["name"])
    print(IDSindicators) # to view the indicator ids and names, remove the "#" at the beginning of the print command
    

('BM.GSR.TOTL.CD', 'Imports of goods, services and primary income (BoP, current US$)')
('BN.CAB.XOKA.CD', 'Current account balance (BoP, current US$)')
('BX.GRT.EXTA.CD.DT', 'Grants, excluding technical cooperation (current US$)')
('BX.GRT.TECH.CD.DT', 'Technical cooperation grants (current US$)')
('BX.GSR.TOTL.CD', 'Exports of goods, services and primary income (BoP, current US$)')
('BX.KLT.DINV.CD.DT', 'Foreign direct investment, net inflows in reporting economy (DRS, current US$)')
('BX.KLT.DREM.CD.DT', 'Primary income on FDI (current US$)')
('BX.PEF.TOTL.CD.DT', 'Portfolio investment, equity (DRS, current US$)')
('BX.TRF.PWKR.CD.DT', 'Personal remittances, received (current US$)')
('DT.AMT.BLAT.CB.CD', 'CB, bilateral (AMT, current US$)')
('DT.AMT.BLAT.CD', 'PPG, bilateral (AMT, current US$)')
('DT.AMT.BLAT.GG.CD', 'GG, bilateral (AMT, current US$)')
('DT.AMT.BLAT.OPS.CD', 'OPS, bilateral (AMT, current US$)')
('DT.AMT.BLAT.PRVG.CD', 'PRVG, bilateral (AMT, current US$)')
('DT.AMT.BLA

### Get indicator definition

In [24]:
# Use the indicator code to define the "indicator" variable
indicator = "DT.DOD.DLXF.CD"

# Parse through the response to get the "sourceNote" or definition for the desired indicator
for dict_entity in indicatorsJSON[1]:
    if dict_entity["id"] == indicator:
        print(dict_entity["sourceNote"])
    else:
        pass

Long-term debt is debt that has an original or extended maturity of more than one year. It has three components: public, publicly guaranteed, and private nonguaranteed debt. Data are in current U.S. dollars.


### Location API Queries

#### Get deptors

In [25]:
# Requesting the locations
dlocations = requests.get("http://api.worldbank.org/v2/sources/6/country?per_page=300&format=JSON")
dlocationsJSON = dlocations.json()

# Parse through the response to see the location IDs and names
dlocations = dlocationsJSON["source"][0]["concept"][0]["variable"]
listLen = int(len(dlocations))

# Create dataframe with location values
df = pd.DataFrame(columns=["id", "value"])     
for i in range(0,listLen):
    code = dlocations[i]["id"]
    name = dlocations[i]["value"]
    df = df.append({"id":code, "value":name}, ignore_index = True)
dlocationsList = df

# See first few items in the dataframe
print(dlocationsList.head(n=10))

    id         value
0  AFG   Afghanistan
1  AGO        Angola
2  ALB       Albania
3  ARG     Argentina
4  ARM       Armenia
5  AZE    Azerbaijan
6  BDI       Burundi
7  BEN         Benin
8  BFA  Burkina Faso
9  BGD    Bangladesh


#### Get creditors

In [26]:
# Requesting the locations
clocations = requests.get("http://api.worldbank.org/v2/sources/6/counterpart-area?per_page=300&format=JSON")
clocationsJSON = clocations.json()

# Parse through the response to see the location IDs and names
clocations = clocationsJSON["source"][0]["concept"][0]["variable"]
listLen = int(len(clocations))

# Create dataframe with location values
df = pd.DataFrame(columns=["id", "value"])     
for i in range(0,listLen):
    code = clocations[i]["id"]
    name = clocations[i]["value"]
    df = df.append({"id":code, "value":name}, ignore_index = True)
clocationsList = df

# See first few items in the dataframe
print(clocationsList.head(n=10))

    id                 value
0  001               Austria
1  002               Belgium
2  003               Denmark
3  004                France
4  005  Germany, Fed.Rep. Of
5  006                 Italy
6  007           Netherlands
7  008                Norway
8  009              Portugal
9  010                Sweden


In [27]:
# Requesting the locations
dlocations = requests.get("http://api.worldbank.org/v2/sources/6/country?per_page=300&format=JSON")
dlocationsJSON = dlocations.json()
dlocationsJSON

{'page': 1,
 'pages': 1,
 'per_page': 300,
 'total': 136,
 'source': [{'id': '6',
   'name': 'International Debt Statistics',
   'concept': [{'id': 'country',
     'name': 'country',
     'variable': [{'id': 'AFG', 'value': 'Afghanistan'},
      {'id': 'AGO', 'value': 'Angola'},
      {'id': 'ALB', 'value': 'Albania'},
      {'id': 'ARG', 'value': 'Argentina'},
      {'id': 'ARM', 'value': 'Armenia'},
      {'id': 'AZE', 'value': 'Azerbaijan'},
      {'id': 'BDI', 'value': 'Burundi'},
      {'id': 'BEN', 'value': 'Benin'},
      {'id': 'BFA', 'value': 'Burkina Faso'},
      {'id': 'BGD', 'value': 'Bangladesh'},
      {'id': 'BGR', 'value': 'Bulgaria'},
      {'id': 'BIH', 'value': 'Bosnia and Herzegovina'},
      {'id': 'BLR', 'value': 'Belarus'},
      {'id': 'BLZ', 'value': 'Belize'},
      {'id': 'BOL', 'value': 'Bolivia'},
      {'id': 'BRA', 'value': 'Brazil'},
      {'id': 'BTN', 'value': 'Bhutan'},
      {'id': 'BWA', 'value': 'Botswana'},
      {'id': 'CAF', 'value': 'Central A

### Part 2

https://worldbank.github.io/debt-data/api-guide/ids-api-guide-python-2.html

In [29]:
# get all indicators for data source 6 (International Debt Statistics)
wbdata.get_indicator(source=6)  

id                   name
-------------------  ---------------------------------------------------------------------------------------------------
BM.GSR.TOTL.CD       Imports of goods, services and primary income (BoP, current US$)
BN.CAB.XOKA.CD       Current account balance (BoP, current US$)
BX.GRT.EXTA.CD.DT    Grants, excluding technical cooperation (current US$)
BX.GRT.TECH.CD.DT    Technical cooperation grants (current US$)
BX.GSR.TOTL.CD       Exports of goods, services and primary income (BoP, current US$)
BX.KLT.DINV.CD.DT    Foreign direct investment, net inflows in reporting economy (DRS, current US$)
BX.KLT.DREM.CD.DT    Primary income on FDI (current US$)
BX.PEF.TOTL.CD.DT    Portfolio investment, equity (DRS, current US$)
BX.TRF.PWKR.CD.DT    Personal remittances, received (current US$)
DT.AMT.BLAT.CB.CD    CB, bilateral (AMT, current US$)
DT.AMT.BLAT.CD       PPG, bilateral (AMT, current US$)
DT.AMT.BLAT.GG.CD    GG, bilateral (AMT, current US$)
DT.AMT.BLAT.OPS.CD   OP

In [30]:
# Selecting the indicator
indicatorSelection = {"DT.DOD.DLXF.CD":"ExternalDebtStock"}

In [31]:
# Select the countries or regions
locationSelection = ["ECA","SSA","SAS","LAC","MNA","EAP"]

In [32]:
# Selecting the time frame
timeSelection = (datetime.datetime(2009, 1, 1), datetime.datetime(2018, 12, 31))

In [38]:
IDS = wbdata.get_source(source_id=6)
IDS

  id  name
----  -----------------------------
   6  International Debt Statistics

In [121]:
wbdata.get_data("DT.DOD.DLXF.CD", country="NER")

[{'indicator': {'id': 'DT.DOD.DLXF.CD',
   'value': 'External debt stocks, long-term (DOD, current US$)'},
  'country': {'id': 'NE', 'value': 'Niger'},
  'countryiso3code': 'NER',
  'date': '2021',
  'value': None,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'DT.DOD.DLXF.CD',
   'value': 'External debt stocks, long-term (DOD, current US$)'},
  'country': {'id': 'NE', 'value': 'Niger'},
  'countryiso3code': 'NER',
  'date': '2020',
  'value': 4031228014.4,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'DT.DOD.DLXF.CD',
   'value': 'External debt stocks, long-term (DOD, current US$)'},
  'country': {'id': 'NE', 'value': 'Niger'},
  'countryiso3code': 'NER',
  'date': '2019',
  'value': 3169734002.9,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'DT.DOD.DLXF.CD',
   'value': 'External debt stocks, long-term (DOD, current US$)'},
  'country': {'id': 'NE', 'value': 'Niger'},
  'countryiso3code': 'NER',
  'dat

In [57]:
wbdata.get_indicator(source=6)

id                   name
-------------------  ---------------------------------------------------------------------------------------------------
BM.GSR.TOTL.CD       Imports of goods, services and primary income (BoP, current US$)
BN.CAB.XOKA.CD       Current account balance (BoP, current US$)
BX.GRT.EXTA.CD.DT    Grants, excluding technical cooperation (current US$)
BX.GRT.TECH.CD.DT    Technical cooperation grants (current US$)
BX.GSR.TOTL.CD       Exports of goods, services and primary income (BoP, current US$)
BX.KLT.DINV.CD.DT    Foreign direct investment, net inflows in reporting economy (DRS, current US$)
BX.KLT.DREM.CD.DT    Primary income on FDI (current US$)
BX.PEF.TOTL.CD.DT    Portfolio investment, equity (DRS, current US$)
BX.TRF.PWKR.CD.DT    Personal remittances, received (current US$)
DT.AMT.BLAT.CB.CD    CB, bilateral (AMT, current US$)
DT.AMT.BLAT.CD       PPG, bilateral (AMT, current US$)
DT.AMT.BLAT.GG.CD    GG, bilateral (AMT, current US$)
DT.AMT.BLAT.OPS.CD   OP

In [81]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [113]:
countries = [i['id'] for i in wbdata.get_country(incomelevel='UMC')]
indicators = {"DT.DOD.DLXF.CD": "debt"}
df = wbdata.get_dataframe(indicators,country=countries, convert_date=True)
df = df.reset_index()
df

,country,date,debt
0,Albania,2021-01-01,NaN
1,Albania,2020-01-01,9.562338e+09
2,Albania,2019-01-01,8.501932e+09
3,Albania,2018-01-01,7.501582e+09
4,Albania,2017-01-01,7.420920e+09
5,Albania,2016-01-01,6.482322e+09
6,Albania,2015-01-01,6.617177e+09
7,Albania,2014-01-01,6.613582e+09
8,Albania,2013-01-01,6.964743e+09
9,Albania,2012-01-01,5.807571e+09


In [115]:
df.country.unique()

array(['Albania', 'Argentina', 'Armenia', 'American Samoa', 'Azerbaijan',
       'Bulgaria', 'Bosnia and Herzegovina', 'Belarus', 'Brazil',
       'Botswana', 'China', 'Colombia', 'Costa Rica', 'Cuba', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Fiji', 'Gabon', 'Georgia',
       'Equatorial Guinea', 'Grenada', 'Guatemala', 'Guyana', 'Iraq',
       'Jamaica', 'Jordan', 'Kazakhstan', 'Lebanon', 'Libya', 'St. Lucia',
       'Moldova', 'Maldives', 'Mexico', 'Marshall Islands',
       'North Macedonia', 'Montenegro', 'Mauritius', 'Malaysia',
       'Namibia', 'Panama', 'Peru', 'Paraguay', 'Romania',
       'Russian Federation', 'Serbia', 'Suriname', 'Thailand',
       'Turkmenistan', 'Tonga', 'Turkey', 'Tuvalu',
       'St. Vincent and the Grenadines', 'Kosovo', 'South Africa'],
      dtype=object)

### Get more dept data

In [118]:
data_date = datetime.datetime(2010, 1, 1), datetime.datetime(2011, 1, 1)                                                                                             
wbdata.get_data("IC.BUS.EASE.XQ", country=["USA", "GBR"], data_date=data_date)  

[{'indicator': {'id': 'IC.BUS.EASE.XQ',
   'value': 'Ease of doing business rank (1=most business-friendly regulations)'},
  'country': {'id': 'GB', 'value': 'United Kingdom'},
  'countryiso3code': 'GBR',
  'date': '2011',
  'value': None,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'IC.BUS.EASE.XQ',
   'value': 'Ease of doing business rank (1=most business-friendly regulations)'},
  'country': {'id': 'GB', 'value': 'United Kingdom'},
  'countryiso3code': 'GBR',
  'date': '2010',
  'value': None,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'IC.BUS.EASE.XQ',
   'value': 'Ease of doing business rank (1=most business-friendly regulations)'},
  'country': {'id': 'US', 'value': 'United States'},
  'countryiso3code': 'USA',
  'date': '2011',
  'value': None,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'IC.BUS.EASE.XQ',
   'value': 'Ease of doing business rank (1=most business-friendly regulations)'},
  'co

In [120]:
wbdata.search_indicators("debt")

id                                        name
----------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------------
5.04.01.01.exdebt                         External debt reporting status
BN.DSR.UNPD.CD                            Debt service not paid (BoP, current US$)
BN.FAC.ARAC.CD                            Debt Service not paid: Arrears Accumulation (BoP, current US$)
DB.DOD.DLXF.CD                            Central Government External Debt (US$)
DE.DOD.DLXF.CD                            Non-financial Pub. Enterprises Ext. Debt (US$)
DG.DOD.MWBG.CD                            Pub. LT Debt, IBRD&IDA (US$, IBRD DRS - end of year)
DG.DOD.OFFL.CD                            Pub. LT Debt, Official Creditors(US$, IBRD DRS)
DG.DOD.PRVT.CD                            Pub. LT Debt, Private Creditors (US$, IBRD DRS)
DG.DOD.TOTL.CD                            Public/Pub. Guar. Long-T